In [3]:
from openalgo import api
import os

OPEN_ALGO_API_KEY = os.getenv('OPEN_ALGO_API_KEY')
# Initialize the API client
client = api(api_key=OPEN_ALGO_API_KEY, host='http://127.0.0.1:5000')

# Fetch historical data for SBIN
df = client.history(
    symbol="RELIANCE",
    exchange="NSE",
    interval="1m",
    start_date="2025-04-30",
    end_date="2025-04-30"
)

df

,close,high,low,open,volume
timestamp,,,,,
2025-04-30 09:15:00+05:30,1398.4,1402.0,1394.2,1402.0,718859
2025-04-30 09:16:00+05:30,1396.8,1399.1,1396.8,1398.5,239568
2025-04-30 09:17:00+05:30,1392.3,1397.4,1392.2,1396.3,143043
2025-04-30 09:18:00+05:30,1394.5,1394.9,1392.0,1392.4,112974
2025-04-30 09:19:00+05:30,1395.3,1395.9,1393.8,1393.9,84258
...,...,...,...,...,...
2025-04-30 15:25:00+05:30,1399.9,1400.5,1399.8,1400.5,267240
2025-04-30 15:26:00+05:30,1396.9,1399.8,1396.9,1399.8,401180
2025-04-30 15:27:00+05:30,1398.1,1398.6,1369.0,1397.1,602763


In [73]:
import math
import pandas as pd

def get_relative_volume_changes(df):
    if isinstance(df, dict) and "candles" in df:
        candles = df["candles"]
    elif isinstance(df, pd.DataFrame):
        dates = df.index.tolist()
        volumes = df["volume"].tolist()
        closes = df["close"].tolist()
        opens = df["open"].tolist()
    elif isinstance(df, list):
        candles = df
    else:
        raise ValueError("Unsupported format for 'df'")

    result = {}
    def sign(x):
        return (x > 0) - (x < 0)
    if 'candles' in locals():
        for i in range(1, len(candles)):
            try:
                date1 = str(candles[i - 1][0])[:10]
                date2 = str(candles[i][0])[:10]
                vol1 = candles[i - 1][5]
                vol2 = candles[i][5]
                key = f"{date2} - {date1}"
                result[key] = round((vol2 - vol1) / vol1, 4)
            except Exception:
                print(f"Error processing candle data at index {i}: {candles[i]}")
                continue
    else:
        for i in range(1, len(volumes)):
            try:
                date1 = str(dates[i - 1])[:10]
                date2 = str(dates[i])[:10]
                vol1 = volumes[i - 1]
                vol2 = volumes[i]
                key = f"{date2} - {date1}"
                diff = vol2 - vol1
                d1= closes[i-1]-opens[i - 1]
                d2= closes[i]-opens[i]
                rg=sign(d1)*sign(d2)
                
                result[f"{date2} - {date1}"] = {
                    'vol1': vol1,
                    'vol2': vol2,
                    'diff': diff,
                    # 'percentage': pct,
                    'd1': d1,
                    'd2': d2,
                    # 'rg': rg
                }

                result[key] = {
                    'vol1': vol1,
                    'vol2': vol2,
                    'diff': diff,
                    'percentage': round(diff / vol1, 4),
                    'd1': d1,
                    'd2': d2,
                    'rg': rg

                }
            except Exception:
                print(f"Error processing volume data at index {i}: {closes[i]}")
                continue

    return result

changes = get_relative_volume_changes(df)



In [75]:

# Convert to DataFrame
table_df = pd.DataFrame.from_dict(changes, orient='index')


# Filter changes with abs(percentage) >= 0.3
filtered_changes = {
    k: v for k, v in changes.items() if abs(v['percentage']) <= 0.03
}

# filter rg with only positive values
filtered_changes = {
    k: v for k, v in filtered_changes.items() if v['rg'] > 0
}

# Convert to DataFrame
table_df = pd.DataFrame.from_dict(filtered_changes, orient='index')

# Optional: Reset index to make the date range a column
table_df.reset_index(inplace=True)
table_df.rename(columns={'index': 'date_range'}, inplace=True)

table_df

,date_range,vol1,vol2,diff,percentage,d1,d2,rg
0,2024-02-05 - 2024-02-04,8814432,9047984,233552,0.0265,-21.75,-14.05,1
1,2024-02-15 - 2024-02-14,10006782,9767498,-239284,-0.0239,-12.75,-15.90,1
2,2024-06-05 - 2024-06-04,17464890,17855722,390832,0.0224,-1.25,-3.40,1
3,2024-06-27 - 2024-06-26,28383030,28957336,574306,0.0202,16.80,34.40,1
4,2024-07-01 - 2024-06-30,7724306,7749246,24940,0.0032,-2.35,-4.30,1
5,2024-07-09 - 2024-07-08,6960002,7134578,174576,0.0251,-7.30,-10.80,1
6,2024-10-03 - 2024-10-02,37757294,37072876,-684418,-0.0181,-29.60,-17.40,1
7,2024-10-29 - 2024-10-28,12008361,11984423,-23938,-0.0020,11.90,8.90,1
8,2025-01-02 - 2025-01-01,15486276,15521102,34826,0.0022,20.55,7.25,1
9,2025-02-26 - 2025-02-24,11552182,11509215,-42967,-0.0037,-7.00,-5.70,1


In [49]:
table_df.to_excel("volume_changes.xlsx", sheet_name="FilteredChanges", index=False)
